In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn import preprocessing as pre
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [2]:
def dataset():
    data = pd.read_csv('final-dataset.csv')
    X = data.drop(columns=['Signal','Label','Hurst Component'])
    y = data['Label']
    feature_names = X.columns
    return X, y, feature_names

In [3]:
def get_feature_names(selector, feature_names):
    mask = selector.get_support() #list of booleans
    new_features = [] # The list of your K best features
    for bool, feature in zip(mask, feature_names):
        if bool:
            new_features.append(feature)
    return new_features

In [4]:
from sklearn.metrics import confusion_matrix
def performance_measures(x_test, y_test):
    y_true = y_test
    y_pred = clf.predict(x_test)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp + fn)
    return sensitivity, specificity

In [5]:
parameters = {'C':range(1,10), 'kernel':('linear', 'poly', 'rbf', 'sigmoid'),'degree':range(2,5),'gamma':('scale','auto')}

In [6]:
def feature_scaling_min_max(X):
    min_max_scaler = pre.MinMaxScaler()
    X = min_max_scaler.fit_transform(X)
    return X

In [7]:
def feature_scaling_basic(X):
    return pre.scale(X)

In [8]:
max_score = 0
best_params = {}
best_k = 2
best_sensitivity = 0
best_specificity = 0
selected_names = []
best_val_score = 0

for k in range(2,10):
    print("passed ", k, "steps")
    X, y, feature_names = dataset()
    X = feature_scaling_min_max(X)
    selector = SelectKBest(chi2, k=k)
    X_new = selector.fit_transform(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=0)
    x_val, x_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)
    clf = SVC(random_state=0)
    clf = GridSearchCV(clf, parameters)
    clf.fit(X_train, y_train)
    val_score = clf.score(x_val, y_val)
    names = get_feature_names(selector, feature_names)
    sensitivity, specificity = performance_measures(x_test, y_test)
    score = clf.score(x_test, y_test)
    if(score > max_score):
        max_score = score
        best_k = k
        best_params = clf.best_params_
        best_val_score = val_score
        selected_names = names
        best_sensitivity = sensitivity
        best_specificity = specificity

print("Test Score is ", max_score)
print("Best params are ", best_params)
print("Best k is ", best_k)
print("Val score is ", best_val_score)
print("Sensitivity ", sensitivity)
print("Specificity ", specificity)
print("names are ", selected_names)

passed  2 steps
passed  3 steps
passed  4 steps
passed  5 steps
passed  6 steps
passed  7 steps
passed  8 steps
passed  9 steps
Test Score is  0.6857142857142857
Best params are  {'C': 5, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best k is  9
Val score is  0.6428571428571429
Sensitivity  0.7403846153846154
Specificity  0.6320754716981132
names are  ['Zero Crossings', 'Frequency (Hz)', 'Delta (µV² /Hz)', 'Alpha (µV² /Hz)', 'Beta (µV² /Hz)', 'Gamma (µV² /Hz)', 'Skewness', 'Hjorth Mobility', 'Spectral Entropy']


In [9]:
max_score = 0
best_params = {}
best_k = 2
best_val_score = 0

for k in range(2,10):
    X, y, feature_names = dataset()
    X = feature_scaling_min_max(X)
    selector = SelectKBest(f_classif, k=k)
    X_new = selector.fit_transform(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=0)
    x_val, x_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)
    clf = SVC(random_state=0)
    clf = GridSearchCV(clf, parameters)
    clf.fit(X_train, y_train)
    val_score = clf.score(x_val, y_val)
    names = get_feature_names(selector, feature_names)
    sensitivity, specificity = performance_measures(x_test, y_test)
    score = clf.score(x_test, y_test)
    if(score > max_score):
        max_score = score
        best_k = k
        best_params = clf.best_params_
        selected_names = names
        best_sensitivity = sensitivity
        best_specificity = specificity
        best_val_score = val_score
    print("Passed ", k, " steps")

print("Test Score is ", max_score)
print("Best params are ", best_params)
print("Best k is ", best_k)
print("Sensitivity ", sensitivity)
print("Specificity ", specificity)
print("names are ", selected_names)
print("Val score is ", best_val_score)

Passed  2  steps
Passed  3  steps
Passed  4  steps
Passed  5  steps
Passed  6  steps
Passed  7  steps
Passed  8  steps
Passed  9  steps
Test Score is  0.719047619047619
Best params are  {'C': 8, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best k is  7
Sensitivity  0.7403846153846154
Specificity  0.6320754716981132
names are  ['Zero Crossings', 'Frequency (Hz)', 'Beta (µV² /Hz)', 'Gamma (µV² /Hz)', 'Skewness', 'Hjorth Mobility', 'Spectral Entropy']
Val score is  0.6476190476190476


In [10]:
max_score = 0
best_params = {}
best_k = 2
best_val_score = 0

for k in range(2,10):
    X, y, feature_names = dataset()
    X = feature_scaling_min_max(X)
    selector = SelectKBest(mutual_info_classif, k=k)
    X_new = selector.fit_transform(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=0)
    x_val, x_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)
    clf = SVC(random_state=0)
    clf = GridSearchCV(clf, parameters)
    clf.fit(X_train, y_train)
    names = get_feature_names(selector, feature_names)
    sensitivity, specificity = performance_measures(x_test, y_test)
    val_score = clf.score(x_val, y_val)
    score = clf.score(x_test, y_test)
    if(score > max_score):
        max_score = score
        best_k = k
        best_params = clf.best_params_
        best_val_score = val_score
        selected_names = names
        best_sensitivity = sensitivity
        best_specificity = specificity
    print("Passed ", k, " steps")

print("Test Score is ", max_score)
print("Best params are ", best_params)
print("Best k is ", best_k)
print("Val score is ", best_val_score)
print("Sensitivity ", sensitivity)
print("Specificity ", specificity)
print("names are ", selected_names)

Passed  2  steps
Passed  3  steps
Passed  4  steps
Passed  5  steps
Passed  6  steps
Passed  7  steps
Passed  8  steps
Passed  9  steps
Test Score is  0.638095238095238
Best params are  {'C': 8, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best k is  6
Val score is  0.6523809523809524
Sensitivity  0.7115384615384616
Specificity  0.44339622641509435
names are  ['RMS (µV)', 'Theta (µV² /Hz)', 'Alpha (µV² /Hz)', 'Beta (µV² /Hz)', 'Gamma (µV² /Hz)', 'Variance']
